In [1]:
import numpy as np
from Gibbs.GibbsSampler import GibbsSampler
from DataGeneration import generate_multiple_series
from Gibbs import Initialize
from Trends import Regressors
import torch
from Gibbs.Parameters import *
from Gibbs import Priors

### Initialize the Gibbs Sampler
Data and weights:

In [2]:
data = generate_multiple_series(150, 120, 10**4, 10**5, "gibbs.csv")
w = np.random.random(120)
w = w/np.sum(w)

Create the Gibbs sampler object:

In [3]:
gibbs = GibbsSampler(data, 16, w)

c:\Users\konto\Documents\Economy-and-Population-Growth\Gibbs\Initialize.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(mat[0], device=device), torch.tensor(factor*Sigma_U, device=device)
c:\Users\konto\Documents\Economy-and-Population-Growth\Gibbs\Parameters.py:64: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3641.)
  self.ws = torch.tensor(np.kron(w.T, I).T, device=device)


### Step 1: Calculate $\{X_i\}_{i=1}^n$

In [4]:
gibbs.step_1()

C:\Users\konto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\functional.py:1816: UserWarning: torch.chain_matmul is deprecated and will be removed in a future PyTorch release. Use torch.linalg.multi_dot instead, which accepts a list of two or more tensors rather than multiple parameters. (Triggered internally at ..\aten\src\ATen\native\LinearAlgebra.cpp:1095.)
  return _VF.chain_matmul(matrices)  # type: ignore[attr-defined]


### Step 2: Calculate $(F, S_m)$:

In [5]:
gibbs.step_2()